In [4]:
pip install pyarrow boto3 pandas chardet

Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
import s3fs
import chardet

# Configuration MinIO
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

input_bucket = 'aymericp/SDV_CNAM'
output_bucket = 'aymericp/SDV_CNAM_parquet'

def list_matching_files(year):
    files = []
    for month in range(1, 13):
        mm = f"{month:02d}"
        for ext in ['csv', 'CSV']:
            key = f"{input_bucket}/NTOT{year}_{mm}.{ext}"
            if fs.exists(key):
                files.append(key)
    return files

def read_csv_from_s3(key):
    # Lire un échantillon de bytes pour détecter l'encodage
    with fs.open(key, 'rb') as f:
        raw_data = f.read(10000)  # Lire les 10 premiers ko
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        confidence = result['confidence']
        print(f"    🔍 Encodage détecté pour {key} : {encoding} (confiance {confidence:.2f})")

        # Relecture complète avec l'encodage détecté
        f.seek(0)
        if not encoding or confidence < 0.7:
            print(f"    ⚠️ Confiance faible ({confidence:.2f}) pour {key}. Forçage ISO-8859-1.")
            encoding = 'ISO-8859-1'
        return pd.read_csv(f, encoding=encoding, sep=';')

# Traitement principal
for year in range(2010, 2025):
    print(f"🔄 Traitement de l'année {year}...")
    files = list_matching_files(year)
    dfs = []
    for file in files:
        print(f"  📥 Lecture : {file}")
        try:
            df = read_csv_from_s3(file)
            dfs.append(df)
        except Exception as e:
            print(f"    ⚠️ Erreur de lecture ({file}) : {e}")

    if dfs:
        df_year = pd.concat(dfs, ignore_index=True)
        output_path = f"{output_bucket}/NTOT{year}.parquet"
        with fs.open(output_path, 'wb') as f:
            df_year.to_parquet(f, index=False)
        print(f"  ✅ Fichier parquet écrit : {output_path}")
    else:
        print(f"  ⚠️ Aucun fichier valide pour {year}")


🔄 Traitement de l'année 2010...
  📥 Lecture : aymericp/SDV_CNAM/NTOT2010_01.csv
    🔍 Encodage détecté pour aymericp/SDV_CNAM/NTOT2010_01.csv : ISO-8859-1 (confiance 0.73)
  📥 Lecture : aymericp/SDV_CNAM/NTOT2010_02.csv
    🔍 Encodage détecté pour aymericp/SDV_CNAM/NTOT2010_02.csv : ISO-8859-1 (confiance 0.73)
  📥 Lecture : aymericp/SDV_CNAM/NTOT2010_03.csv
    🔍 Encodage détecté pour aymericp/SDV_CNAM/NTOT2010_03.csv : ISO-8859-1 (confiance 0.73)
  📥 Lecture : aymericp/SDV_CNAM/NTOT2010_04.csv
    🔍 Encodage détecté pour aymericp/SDV_CNAM/NTOT2010_04.csv : ISO-8859-1 (confiance 0.73)
  📥 Lecture : aymericp/SDV_CNAM/NTOT2010_05.csv
    🔍 Encodage détecté pour aymericp/SDV_CNAM/NTOT2010_05.csv : ISO-8859-1 (confiance 0.73)
  📥 Lecture : aymericp/SDV_CNAM/NTOT2010_06.csv
    🔍 Encodage détecté pour aymericp/SDV_CNAM/NTOT2010_06.csv : ISO-8859-1 (confiance 0.73)
  📥 Lecture : aymericp/SDV_CNAM/NTOT2010_07.csv
    🔍 Encodage détecté pour aymericp/SDV_CNAM/NTOT2010_07.csv : ISO-8859-1 (conf

In [10]:
import numpy as np

fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"})

input_bucket = 'aymericp/SDV_CNAM_parquet'
output_bucket = 'aymericp/SDV_CNAM_parquet_float'  # nouveau dossier pour ne pas écraser

def convert_french_number_to_float(series):
    series = series.replace(['None', 'none', 'NA', 'na', 'nan', 'NaN'], np.nan)
    return series.str.replace('.', '', regex=False).str.replace(',', '.', regex=False).astype(float)

cols_to_convert = ['rem_mon', 'rec_mon', 'dep_mon', 'act_coe']

parquet_files = fs.ls(input_bucket)

for file_path in parquet_files:
    if file_path.endswith('.parquet'):
        print(f"Traitement de {file_path}")
        try:
            with fs.open(file_path, 'rb') as f:
                df = pd.read_parquet(f)

            for col in cols_to_convert:
                if col in df.columns:
                    # Convertir en string, gérer NaN puis float
                    df[col] = convert_french_number_to_float(df[col].astype(str))

            output_path = file_path.replace(input_bucket, output_bucket)
            output_dir = '/'.join(output_path.split('/')[:-1])
            if not fs.exists(output_dir):
                fs.makedirs(output_dir)

            with fs.open(output_path, 'wb') as f_out:
                df.to_parquet(f_out, index=False)

            print(f"✅ Fichier écrit : {output_path}")

        except Exception as e:
            print(f"⚠️ Erreur sur {file_path} : {e}")



Traitement de aymericp/SDV_CNAM_parquet/NTOT2010.parquet
✅ Fichier écrit : aymericp/SDV_CNAM_parquet_float/NTOT2010.parquet
Traitement de aymericp/SDV_CNAM_parquet/NTOT2011.parquet
✅ Fichier écrit : aymericp/SDV_CNAM_parquet_float/NTOT2011.parquet
Traitement de aymericp/SDV_CNAM_parquet/NTOT2012.parquet
✅ Fichier écrit : aymericp/SDV_CNAM_parquet_float/NTOT2012.parquet
Traitement de aymericp/SDV_CNAM_parquet/NTOT2013.parquet
✅ Fichier écrit : aymericp/SDV_CNAM_parquet_float/NTOT2013.parquet
Traitement de aymericp/SDV_CNAM_parquet/NTOT2014.parquet
✅ Fichier écrit : aymericp/SDV_CNAM_parquet_float/NTOT2014.parquet
Traitement de aymericp/SDV_CNAM_parquet/NTOT2015.parquet
✅ Fichier écrit : aymericp/SDV_CNAM_parquet_float/NTOT2015.parquet
Traitement de aymericp/SDV_CNAM_parquet/NTOT2016.parquet
✅ Fichier écrit : aymericp/SDV_CNAM_parquet_float/NTOT2016.parquet
Traitement de aymericp/SDV_CNAM_parquet/NTOT2017.parquet
✅ Fichier écrit : aymericp/SDV_CNAM_parquet_float/NTOT2017.parquet
Traiteme